In [11]:
import pandas as pd
import numpy as np
import datetime
import pickle
from string import Template
import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
with open('MTA_Apr_June_with_weeks.pickle', 'rb') as f:
    data = pickle.load(f)

In [13]:
# remove white space from columns
data.columns = [column.strip() for column in data.columns]
data.rename(columns={'week':'WEEK'}, inplace=True)
data.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,WEEK
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/21/2018,00:00:00,REGULAR,6590024,2232650,1
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/21/2018,04:00:00,REGULAR,6590038,2232663,1
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/21/2018,08:00:00,REGULAR,6590050,2232693,1
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/21/2018,12:00:00,REGULAR,6590131,2232766,1
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/21/2018,16:00:00,REGULAR,6590350,2232816,1


In [14]:
#data.info()
data[data['STATION'] == '23 ST'].head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,WEEK
3948,A030,R083,01-00-00,23 ST,NRW,BMT,04/21/2018,00:00:00,REGULAR,6571203,4200488,1
3949,A030,R083,01-00-00,23 ST,NRW,BMT,04/21/2018,04:00:00,REGULAR,6571276,4200508,1
3950,A030,R083,01-00-00,23 ST,NRW,BMT,04/21/2018,08:00:00,REGULAR,6571301,4200573,1
3951,A030,R083,01-00-00,23 ST,NRW,BMT,04/21/2018,12:00:00,REGULAR,6571447,4200832,1
3952,A030,R083,01-00-00,23 ST,NRW,BMT,04/21/2018,16:00:00,REGULAR,6571892,4201126,1


In [15]:
data['TIME'] = pd.to_datetime(data['TIME'], infer_datetime_format=True, cache=True).apply(lambda x: x.time())
data.TIME.value_counts()

00:00:00    170048
04:00:00    170046
20:00:00    170017
08:00:00    169996
16:00:00    169992
12:00:00    169854
01:00:00    120950
05:00:00    120928
09:00:00    120830
21:00:00    120828
13:00:00    120815
17:00:00    120814
22:00:00      7694
18:00:00      7693
10:00:00      7692
02:00:00      7691
14:00:00      7684
06:00:00      7682
23:00:00      2731
11:00:00      2731
15:00:00      2729
03:00:00      2727
19:00:00      2726
07:00:00      2716
00:22:00      2594
12:22:00      2591
04:22:00      2590
20:22:00      2590
08:22:00      2590
16:22:00      2577
             ...  
11:24:04         1
12:55:38         1
03:44:11         1
04:22:45         1
15:55:40         1
10:53:55         1
11:06:09         1
03:46:28         1
05:10:36         1
19:20:33         1
09:27:53         1
10:43:59         1
19:35:39         1
21:47:37         1
01:03:32         1
22:37:31         1
04:05:16         1
01:49:13         1
07:52:49         1
11:15:43         1
22:14:04         1
17:42:34    

In [16]:
#Select data for 12:00 pm-9:00 pm, when most tech-employees and potential donors might be on the streets
data = data[(data['TIME']>pd.to_datetime('12:00:00').time()) & (data['TIME']<pd.to_datetime('21:00:00').time())]

In [17]:
data.shape

(665606, 12)

In [18]:
#function to calculate the difference between a series max and min values.
#will be applied on unique turnstile_id + day-level
def difference(srs):
    return srs.max() - srs.min()

#Identifying and grouping by unique turnstiles, taking the daily max-daily min for that turnstile:

In [19]:
turnstile_col_names =['STATION','C/A','UNIT','SCP']

by_turnstile_date =  data.groupby(by=turnstile_col_names+['WEEK']+['DATE'], as_index=False).agg(difference)
by_turnstile_date['turnstile_id'] = by_turnstile_date.apply(lambda x: str(x['STATION']+'_'+x['C/A']+'_'+x['UNIT']+'_'+x['SCP']), axis=1)

In [20]:
by_turnstile_date['ENTRIES'].describe()
#plt.hist(by_station_date['ENTRIES'][by_station_date['ENTRIES']>861])

count    3.285520e+05
mean     1.927503e+04
std      4.988357e+06
min      0.000000e+00
25%      6.300000e+01
50%      2.110000e+02
75%      4.410000e+02
max      2.113960e+09
Name: ENTRIES, dtype: float64

#turnstiles with 1 really high day's entries probably are showing data anomalies:

In [21]:
by_turnstile_date[by_turnstile_date['ENTRIES']>4000].turnstile_id.value_counts()

JOURNAL SQUARE_PTH03_R552_00-00-09     3
SIMPSON ST_R317_R408_01-00-00          2
PATH NEW WTC_PTH22_R540_00-01-00       2
BRONX PARK EAST_R326_R389_00-00-02     2
AVENUE U_D015_R396_00-06-00            1
47-50 STS ROCK_N501_R020_01-06-01      1
JOURNAL SQUARE_PTH03_R552_00-01-02     1
KINGSBRIDGE RD_N221_R155_00-00-00      1
TREMONT AV_N213_R154_00-06-01          1
CANAL ST_A046_R463_00-06-05            1
CITY / BUS_PTH07_R550_00-00-04         1
36 AV_R511_R091_00-00-00               1
PATH NEW WTC_PTH22_R540_00-03-02       1
THIRTY THIRD ST_PTH17_R541_01-00-07    1
NEWARK HM HE_PTH20_R549_03-00-00       1
NEWARK BM BW_PTH18_R549_01-01-01       1
BROOKLYN BRIDGE_R210_R044_00-03-03     1
CANAL ST_R119_R320_00-00-02            1
BRONX PARK EAST_R326_R389_00-00-01     1
FLUSHING-MAIN_R533_R055_00-03-07       1
68ST-HUNTER CO_R246_R177_00-03-06      1
EXCHANGE PLACE_PTH05_R543_00-04-04     1
JFK JAMAICA CT1_JFK03_R536_00-00-05    1
30 AV_R513_R093_00-03-01               1
190 ST_N006A_R28

#
#identify outlier values of 'ENTRIES' for Turnstile/Date

In [22]:
high_turnstile_count = dict(by_turnstile_date[by_turnstile_date['ENTRIES']>4000].turnstile_id.value_counts())
high_turnstile_count = {key : val for key,val in high_turnstile_count.items() if val<2}
high_turnstile_count

{'190 ST_N006A_R280_00-00-00': 1,
 '30 AV_R513_R093_00-03-01': 1,
 '30 AV_R513_R093_00-03-02': 1,
 '34 ST-PENN STA_N071_R013_00-06-00': 1,
 '36 AV_R511_R091_00-00-00': 1,
 '47-50 STS ROCK_N501_R020_01-06-01': 1,
 '68ST-HUNTER CO_R246_R177_00-03-06': 1,
 'AVENUE U_D015_R396_00-06-00': 1,
 'BRONX PARK EAST_R326_R389_00-00-01': 1,
 'BROOKLYN BRIDGE_R210_R044_00-03-03': 1,
 'BURNSIDE AV_R287_R244_00-05-00': 1,
 'CANAL ST_A046_R463_00-06-05': 1,
 'CANAL ST_R119_R320_00-00-02': 1,
 'CITY / BUS_PTH07_R550_00-00-04': 1,
 'EXCHANGE PLACE_PTH05_R543_00-04-04': 1,
 'FLUSHING-MAIN_R533_R055_00-03-07': 1,
 'GRD CNTRL-42 ST_R237B_R047_01-00-02': 1,
 'JFK JAMAICA CT1_JFK03_R536_00-00-05': 1,
 'JOURNAL SQUARE_PTH03_R552_00-01-02': 1,
 'KINGSBRIDGE RD_N221_R155_00-00-00': 1,
 'LEXINGTON AV/53_N305A_R016_00-00-02': 1,
 'NEWARK BM BW_PTH18_R549_01-01-01': 1,
 'NEWARK HM HE_PTH20_R549_03-00-00': 1,
 'PATH NEW WTC_PTH22_R540_00-00-02': 1,
 'PATH NEW WTC_PTH22_R540_00-03-02': 1,
 'THIRTY THIRD ST_PTH17_R541

In [23]:
#identify outlier values of 'ENTRIES' for turnstile/Date pairs and replace them with weekly averages for the turnstile
avg_turnstile_per_day = by_turnstile_date.groupby(by=['STATION','turnstile_id','WEEK'], as_index=False).mean().drop(columns='EXITS')
avg_turnstile_per_day.rename(columns={'ENTRIES':'AVG_REPLACEMENT'}, inplace=True)

by_turnstile_date_anomalies = by_turnstile_date[(by_turnstile_date['ENTRIES']>5000) & (by_turnstile_date['turnstile_id'].isin(high_turnstile_count.keys()))]
by_turnstile_date_anomalies = by_turnstile_date_anomalies.merge(right=avg_turnstile_per_day, how='left')
by_turnstile_date_anomalies['ENTRIES'] = by_turnstile_date_anomalies['AVG_REPLACEMENT']
by_turnstile_date_anomalies.drop(columns=['AVG_REPLACEMENT'], inplace=True)

by_turnstile_date = pd.concat([by_turnstile_date[(by_turnstile_date['ENTRIES']<=30000) | ~(by_turnstile_date['turnstile_id'].isin(high_turnstile_count.keys()))], by_turnstile_date_anomalies])

by_turnstile_date.ENTRIES.describe()

count    3.285550e+05
mean     4.968493e+03
std      1.004774e+06
min      0.000000e+00
25%      6.300000e+01
50%      2.110000e+02
75%      4.410000e+02
max      3.019948e+08
Name: ENTRIES, dtype: float64

In [24]:
by_station_date = by_turnstile_date.groupby(by=['STATION','WEEK', 'DATE'], as_index=False).sum()
by_station_date.ENTRIES.describe()

count    2.621000e+04
mean     6.228246e+04
std      3.557157e+06
min      0.000000e+00
25%      7.400000e+02
50%      1.642500e+03
75%      3.761750e+03
max      3.020204e+08
Name: ENTRIES, dtype: float64

In [25]:
#stations with 1 really high day's entries probably are showing data anomalies.
by_station_date[by_station_date['ENTRIES']>30000].STATION.value_counts()

GRD CNTRL-42 ST    50
TIMES SQ-42 ST     49
34 ST-HERALD SQ    49
34 ST-PENN STA     49
23 ST              49
FULTON ST          48
47-50 STS ROCK     48
CHAMBERS ST        35
59 ST COLUMBUS     31
PATH NEW WTC       30
59 ST              18
14 ST-UNION SQ     12
BRONX PARK EAST     3
JOURNAL SQUARE      3
SIMPSON ST          2
CANAL ST            2
EXCHANGE PLACE      1
JFK JAMAICA CT1     1
FLUSHING-MAIN       1
TREMONT AV          1
AVENUE U            1
30 AV               1
TWENTY THIRD ST     1
KINGSBRIDGE RD      1
36 AV               1
LEXINGTON AV/53     1
190 ST              1
BROOKLYN BRIDGE     1
BURNSIDE AV         1
Name: STATION, dtype: int64

In [26]:
#identify outlier values of 'ENTRIES' for Station/Date pairs and replace them with weekly averages for the station
high_day_count = dict(by_station_date[by_station_date['ENTRIES']>30000].STATION.value_counts())
high_day_count = {key : val for key,val in high_day_count.items() if val<2}
high_day_count

{'190 ST': 1,
 '30 AV': 1,
 '36 AV': 1,
 'AVENUE U': 1,
 'BROOKLYN BRIDGE': 1,
 'BURNSIDE AV': 1,
 'EXCHANGE PLACE': 1,
 'FLUSHING-MAIN': 1,
 'JFK JAMAICA CT1': 1,
 'KINGSBRIDGE RD': 1,
 'LEXINGTON AV/53': 1,
 'TREMONT AV': 1,
 'TWENTY THIRD ST': 1}

In [27]:
#identify outlier values of 'ENTRIES' for Station/Date pairs and replace them with weekly averages for the station
avg_day_per_week = by_station_date.groupby(by=['STATION','WEEK'], as_index=False).mean().drop(columns='EXITS')
avg_day_per_week.rename(columns={'ENTRIES':'AVG_REPLACEMENT'}, inplace=True)

by_station_date_anomalies = by_station_date[(by_station_date['ENTRIES']>30000) & (by_station_date['STATION'].isin(high_day_count.keys()))]
by_station_date_anomalies = by_station_date_anomalies.merge(right=avg_day_per_week, how='left')
by_station_date_anomalies['ENTRIES'] = by_station_date_anomalies['AVG_REPLACEMENT']
by_station_date_anomalies.drop(columns=['AVG_REPLACEMENT'], inplace=True)

by_station_date = pd.concat([by_station_date[(by_station_date['ENTRIES']<=30000) | ~(by_station_date['STATION'].isin(high_day_count.keys()))], by_station_date_anomalies])

by_station_date.ENTRIES.describe()

count    2.621000e+04
mean     5.098164e+04
std      3.270387e+06
min      0.000000e+00
25%      7.400000e+02
50%      1.642500e+03
75%      3.761750e+03
max      3.020204e+08
Name: ENTRIES, dtype: float64

In [28]:
by_station_date.head()

,STATION,WEEK,DATE,ENTRIES,EXITS
0,1 AV,1,04/21/2018,5443.0,6501
1,1 AV,1,04/22/2018,4431.0,5024
2,1 AV,1,04/23/2018,6636.0,6370
3,1 AV,1,04/24/2018,6958.0,6691
4,1 AV,1,04/25/2018,6779.0,6832


In [29]:
by_station_date.groupby(by=['STATION','WEEK'], as_index=False).count().ENTRIES.unique()

array([7, 6, 3, 5, 4])

#drop station/week combos with fewer than 7 days per week observed

In [30]:
station_date_counts = by_station_date.groupby(by=['STATION','WEEK'], as_index=False).count()
station_date_counts = station_date_counts[station_date_counts.ENTRIES < 7]
station_date_counts.drop(columns=['ENTRIES','EXITS','DATE'], inplace=True)
by_station_date = by_station_date.merge(right=station_date_counts, on=['STATION','WEEK'], how='left', indicator = True)
by_station_date = by_station_date[by_station_date._merge == 'left_only']
by_station_date.drop(columns=['_merge'], inplace=True)

In [31]:
by_station_date.ENTRIES.describe()

count    2.613800e+04
mean     5.099010e+04
std      3.274855e+06
min      0.000000e+00
25%      7.410000e+02
50%      1.647000e+03
75%      3.770000e+03
max      3.020204e+08
Name: ENTRIES, dtype: float64

In [32]:
with open('by_station_date_cleaned.pickle','wb') as f:
    pickle.dump(by_station_date,f)

In [33]:
by_station_week =  by_station_date.groupby(by=['STATION', 'WEEK'], as_index=False).sum().drop(columns=['EXITS'])
by_station_week.head()

,STATION,WEEK,ENTRIES
0,1 AV,1,44491.0
1,1 AV,2,45225.0
2,1 AV,3,44624.0
3,1 AV,4,43092.0
4,1 AV,5,44693.0


In [34]:
by_station_week.ENTRIES.describe()

count    3.734000e+03
mean     3.569307e+05
std      8.660922e+06
min      0.000000e+00
25%      5.599250e+03
50%      1.223050e+04
75%      2.766100e+04
max      3.021631e+08
Name: ENTRIES, dtype: float64

In [35]:
by_station_week[by_station_week['ENTRIES']>500000]

,STATION,WEEK,ENTRIES
356,190 ST,8,2.922529e+07
1634,BRONX PARK EAST,6,1.504298e+08
1635,BRONX PARK EAST,7,5.114118e+07
1638,BRONX PARK EAST,10,1.408625e+08
1679,BURNSIDE AV,1,2.400488e+06
1707,CANAL ST,9,3.021631e+08
2300,GRD CNTRL-42 ST,5,6.130219e+05
2550,JOURNAL SQUARE,5,2.619001e+06
2605,KINGSBRIDGE RD,10,1.434453e+07
3350,SIMPSON ST,5,2.673926e+08


In [36]:
#get rid of anomalously high weeks
by_station_week = by_station_week[by_station_week['ENTRIES']<500000]

In [37]:
weekly_avg_by_station = by_station_week.groupby(by="STATION", as_index=False).mean().drop(columns=['WEEK'])
weekly_avg_by_station["mean_percentage"] = (weekly_avg_by_station["ENTRIES"]/weekly_avg_by_station["ENTRIES"].sum())*100
weekly_avg_by_station = weekly_avg_by_station.sort_values(by='ENTRIES', ascending=False)
weekly_avg_by_station.head(10)

,STATION,ENTRIES,mean_percentage
58,34 ST-HERALD SQ,309020.600000,3.002723
232,GRD CNTRL-42 ST,274303.777778,2.665383
60,34 ST-PENN STA,271529.728571,2.638428
45,23 ST,260472.300000,2.530984
352,TIMES SQ-42 ST,256701.500000,2.494343
225,FULTON ST,195180.400000,1.896549
71,47-50 STS ROCK,192684.514286,1.872297
14,14 ST-UNION SQ,181547.400000,1.764078
85,59 ST COLUMBUS,179486.500000,1.744053
314,PATH NEW WTC,177153.585714,1.721384


In [38]:
with open('weekly_avg_by_station_clean.pickle', 'wb') as to_write:
    pickle.dump(weekly_avg_by_station, to_write)

In [39]:
weekly_avg_by_station.head(10)

,STATION,ENTRIES,mean_percentage
58,34 ST-HERALD SQ,309020.600000,3.002723
232,GRD CNTRL-42 ST,274303.777778,2.665383
60,34 ST-PENN STA,271529.728571,2.638428
45,23 ST,260472.300000,2.530984
352,TIMES SQ-42 ST,256701.500000,2.494343
225,FULTON ST,195180.400000,1.896549
71,47-50 STS ROCK,192684.514286,1.872297
14,14 ST-UNION SQ,181547.400000,1.764078
85,59 ST COLUMBUS,179486.500000,1.744053
314,PATH NEW WTC,177153.585714,1.721384


In [40]:
weekly_avg_by_station.head(10).mean_percentage.sum()

22.33022098687372